In [1]:
#@title parameters

#model_name = "bert-base-uncased"
#model_name = "roberta-base"
model_name = "microsoft/deberta-base"

random_seed = 42
num_workers = 0

batch_size = 8
learning_rate = 1e-5
epochs = 10

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 1.6 MB/s eta 0:00:00


In [5]:
import torch
import torch.nn as nn
import pandas as pd
from transformers import  AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.optim as optim
import json

In [6]:
print(torch.cuda.is_available())
torch.cuda.set_device(0)
device = torch.device("cuda")

if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


True


In [ ]:
!gdown 1g11MkyTQcuaPHfpFrTNxRw4qzbxiiHWJ #download tran.jsonl
!gdown 1h_iDzd-uyOFoxe9n3BH6K9Xc4O77VhCB #validation.jsonl

Downloading...
From: https://drive.google.com/uc?id=1g11MkyTQcuaPHfpFrTNxRw4qzbxiiHWJ
To: /content/train.jsonl
100% 19.1M/19.1M [00:00<00:00, 101MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1h_iDzd-uyOFoxe9n3BH6K9Xc4O77VhCB
To: /content/validation.jsonl
100% 5.02M/5.02M [00:00<00:00, 189MB/s]


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 3)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
path = "/content/"
train = pd.read_json(path + "train.jsonl", lines=True)
validation = pd.read_json(path + "validation.jsonl", lines=True)
train

,uuid,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags
0,0af11f6b-c889-4520-9372-66ba25cb7657,532quh,"[Wes Welker Wanted Dinner With Tom Brady, But ...",reddit,[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,","[http://pixel.wp.com/b.gif?v=noscript, http://...",http://nesn.com/2016/09/wes-welker-wanted-dinn...,"{'source': 'anonymized', 'humanSpoiler': 'They...",[how about that morning we go throw?],"[[[3, 151], [3, 186]]]",[passage]
1,b1a1f63d-8853-4a11-89e8-6b2952a393ec,411701128456593408,[NASA sets date for full recovery of ozone hole],Twitter,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1cH672Z,"{'source': 'anonymized', 'humanSpoiler': '2070...",[2070],"[[[0, 0], [0, 4]]]",[phrase]
2,008b7b19-0445-4e16-8f9e-075b73f80ca4,380537005123190784,[This is what makes employees happy -- and it'...,Twitter,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[http://i.huffpost.com/gen/1359674/images/o-HA...,http://huff.to/1epfeaw,"{'source': 'anonymized', 'humanSpoiler': 'Inte...",[intellectual stimulation],"[[[1, 186], [1, 210]]]",[phrase]
3,31ecf93c-3e21-4c80-949b-aa549a046b93,844567852531286016,[Passion is overrated — 7 work habits you need...,Twitter,"[It’s common wisdom. Near gospel really, and n...","‘Follow your passion’ is wrong, here are 7 hab...",There's a lot more to work that loving your job,"business, work-life, careers",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[Purpose connects us to something bigger and i...,"[[[11, 25], [11, 101]], [[17, 56], [17, 85]], ...",[multi]
4,31b108a3-c828-421a-a4b9-cf651e9ac859,814186311573766144,[The perfect way to cook rice so that it's per...,Twitter,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[in a rice cooker],"[[[5, 60], [5, 76]]]",[phrase]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,92578045-699f-4957-a3c5-cff2c3874dae,591979258442027008,[Has Facebook's video explosion completely sha...,Twitter,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[http://djcs-prod.s3.amazonaws.com/wsjfrontpag...,http://on.wsj.com/1GfyUqz,"{'source': 'anonymized', 'humanSpoiler': 'No.'...",[it hasn’t necessarily taken the wind out of Y...,"[[[7, 50], [7, 118]]]",[passage]
3196,51682121-df0b-4289-a95f-e1bc3d181306,881531941974661_902284433232745,[Cop Is Eating At A Chili's When Teen Hands Hi...,Facebook,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,[https://sbly-web-prod-shareably.netdna-ssl.co...,http://shareably.net/officer-receives-touching...,"{'source': 'anonymized', 'humanSpoiler': 'The ...","[It read, ""Thanks for keeping us safe.""]","[[[0, 317], [0, 355]]]",[passage]
3197,9c45ca67-38c4-47b4-aa0d-48434bae09fc,837356193576333314,[5 popular myths about visible signs of aging ...,Twitter,"[Obama looks decades younger already, but what...",5 popular myths about visible signs of aging t...,Dozens of anti-wrinkle products and creams are...,,None,https://www.businessinsider.in/5-popular-myths...,"{'source': 'anonymized', 'humanSpoiler': None,...",[1. Anti-wrinkle creams will er

In [ ]:
def delete_brackets_tags(dataframe):
    for index, data_row in dataframe.iterrows():
            data_row["tags"] = data_row["tags"][0]
    return dataframe

train = delete_brackets_tags(train)
validation = delete_brackets_tags(validation)

In [ ]:
def data_processing(dataframe):
    ret = []
    for _, i in dataframe.iterrows():
        post_words = i['postText'][0].split(' ')
        if post_words[-1][-1] not in '!?.':
            post_words[-1] += '.'

        target_paragraphs = i['targetParagraphs']
        if len(target_paragraphs) > 2:
            target_paragraphs = target_paragraphs[1:]

        target_paragraphs_words = []
        for paragraph in target_paragraphs:
            paragraph_words = paragraph.split(' ')
            target_paragraphs_words.extend(paragraph_words)
            if len(target_paragraphs_words) >= 201:
                target_paragraphs_words = target_paragraphs_words[:201]
                break

        final_string = ' '.join(post_words) + ' ' + ' '.join(target_paragraphs_words)
        ret += [{'text': final_string, 'uuid': i['uuid']}]

    return pd.DataFrame(ret)

train_data = data_processing(train)
validation_data = data_processing(validation)

labels_idx = {"multi": 0,	"passage": 1, "phrase": 2}


train_labels = [labels_idx[tag] for tag in train.tags]
validation_labels = [labels_idx[tag] for tag in validation.tags]

In [ ]:
train_tokens = tokenizer(train_data.text.tolist(), padding=True, truncation=True, max_length=512,
                              return_tensors="pt")
validation_tokens = tokenizer(validation_data.text.tolist(), padding=True, truncation=True, max_length=512,
                              return_tensors="pt")

In [ ]:
train_inputs = train_tokens["input_ids"].clone().detach()
train_masks = train_tokens["attention_mask"].clone().detach()
train_labels = torch.tensor(train_labels)

val_inputs = validation_tokens["input_ids"].clone().detach()
val_masks = validation_tokens["attention_mask"].clone().detach()
val_labels = torch.tensor(validation_labels)


train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size, num_workers=num_workers)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size, num_workers=num_workers)


In [ ]:
optimizer = optim.AdamW(model.parameters(), lr = learning_rate)

model = model.to(device)

In [ ]:
def train_epoch(dataloader, model):
    model.train()
    running_loss = 0.0
    s = 0
    for step, batch  in enumerate(dataloader, 0):
        input_id, mask, labels = batch
        input_id, mask, labels = input_id.to(device), mask.to(device), labels.to(device)
        # zero the parameter gradients
        model.zero_grad()

        # forward + backward + optimize
        preds = model(input_id, attention_mask = mask, labels=labels)
        # compute the loss between actual and predicted values
        loss = preds[0]
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        s+=1
    return running_loss / s

def evaluate(dataloader, model):
    model.eval()
    with torch.no_grad():
        good = 0
        all = 0
        for step, batch  in enumerate(dataloader, 0):
            sent_id, mask, labels = batch
            preds = model(sent_id.to(device), mask.to(device))
            preds = np.argmax(preds.logits.cpu(), axis=1).tolist()

            for i in range(len(preds)):
                if preds[i] == labels[i]:
                    good +=1
                all +=1
    return good/all

In [ ]:
for epoch in range(epochs):  # loop over the dataset multiple times
    for phase in ["train", "eval"]:
        if phase == "train":
            epoch_loss = train_epoch(train_dataloader, model)
            print('[%d/10] loss: %.10f' % (epoch+1 ,  epoch_loss))
        elif phase == "eval":
            eval_acc = evaluate(val_dataloader, model)
            print('[%d/10] accuracy on eval set: %.10f' % (epoch+1 ,  eval_acc))
   
    model.save_pretrained(f"/content/{model_name}-{eval_acc}")
print('Finished Training')


[1/10] loss: 1.0228781268
[1/10] accuracy on eval set: 0.5050000000
[2/10] loss: 0.9267705923
[2/10] accuracy on eval set: 0.6112500000
[3/10] loss: 0.7455242936
[3/10] accuracy on eval set: 0.6662500000
[4/10] loss: 0.5160093827
[4/10] accuracy on eval set: 0.6750000000
[5/10] loss: 0.2822443391
[5/10] accuracy on eval set: 0.6825000000
[6/10] loss: 0.1683545157
[6/10] accuracy on eval set: 0.6612500000
[7/10] loss: 0.0843735283
[7/10] accuracy on eval set: 0.6737500000
[8/10] loss: 0.0486120276
[8/10] accuracy on eval set: 0.6600000000
[9/10] loss: 0.0356756165
[9/10] accuracy on eval set: 0.6587500000
[10/10] loss: 0.0323740523
[10/10] accuracy on eval set: 0.6500000000
Finished Training


In [ ]:
model_name

'bert-base-uncased'

In [ ]:
#tokenizer.save_pretrained("/content/bert-tokenizer")
#!zip -r /content/bert.zip /content/bert-base-uncased-0.6825/
#!zip -r /content/bert-tokenizer.zip /content/bert-tokenizer/

  adding: content/bert-base-uncased-0.6825/ (stored 0%)
  adding: content/bert-base-uncased-0.6825/pytorch_model.bin (deflated 7%)
  adding: content/bert-base-uncased-0.6825/config.json (deflated 51%)
  adding: content/bert-tokenizer/ (stored 0%)
  adding: content/bert-tokenizer/special_tokens_map.json (deflated 42%)
  adding: content/bert-tokenizer/vocab.txt (deflated 53%)
  adding: content/bert-tokenizer/tokenizer_config.json (deflated 41%)
  adding: content/bert-tokenizer/tokenizer.json (deflated 71%)
